In [1]:
import ROOT
from ROOT import RDataFrame, TCanvas
%jsroot on

Welcome to JupyROOT 6.24/02


In [2]:
d = RDataFrame('events', 'ttbar.root')

In [3]:
d = d.Define('electron_pt_mask', 'electron_pt>25').Define('muon_pt_mask', 'muon_pt>25').Define('jet_pt_mask', 'jet_pt>25')\
     .Filter('Sum(electron_pt_mask) + Sum(muon_pt_mask) == 1')\
     .Filter('Sum(jet_pt_mask) >= 4')\
     .Filter('Sum(jet_btag[jet_pt_mask]>=0.5)>=2')            
d.Count().GetValue()     

8394

In [4]:
jet_p4 = d.Define("jet_p4", 
    "ROOT::VecOps::Construct<ROOT::Math::PxPyPzMVector>(jet_px[jet_pt_mask], jet_py[jet_pt_mask], jet_pz[jet_pt_mask], jet_mass[jet_pt_mask])"
)

In [5]:
trijet = jet_p4.Define('trijet', 
    'ROOT::VecOps::Combinations(jet_pt[jet_pt_mask],3)'
).Define('ntrijet', 'trijet[0].size()')

In [6]:
trijet_p4 = trijet.Define('trijet_p4', 
                      'ROOT::VecOps::RVec<ROOT::Math::PxPyPzMVector> trijet_p4(ntrijet);'              +\
                      'for (int i = 0; i < ntrijet; ++i) {'                                            +\
                          'int j1 = trijet[0][i]; int j2 = trijet[1][i]; int j3 = trijet[2][i];'       +\
                          'trijet_p4[i] = jet_p4[j1] + jet_p4[j2] + jet_p4[j3];'                       +\
                      '}'                                                                              +\
                      'return trijet_p4;'                                                                                                                          
                     )

In [15]:
#TODO  implement references
trijet_pt = trijet_p4.Define('trijet_pt', 
        'return (ROOT::VecOps::Map(trijet_p4, [](ROOT::Math::PxPyPzMVector v) { return v.Pt(); }))'
                            )

In [16]:
trijet_pt_btag = trijet_pt.Define('trijet_btag', 
                                  'ROOT::VecOps::RVec<bool> btag(ntrijet);'                                   +\
                                  'for (int i = 0; i < ntrijet; ++i) {'                                       +\
                                   'int j1 = trijet[0][i]; int j2 = trijet[1][i]; int j3 = trijet[2][i];'     +\
                                   'btag[i]=std::max({jet_btag[j1], jet_btag[j2], jet_btag[j3]})>0.5;'        +\
                                  '}'                                                                         +\
                                  'return btag;'
                            )

In [56]:
# trijet_mass=trijet_pt_btag.Define('trijet_mass',
#                                  'int ind = ROOT::VecOps::ArgMax(trijet_pt[trijet_btag]);'                    +\
#                                  'return trijet_p4[ind].M();')

In [59]:
trijet_mass=trijet_pt_btag.Define('trijet_mass',
                                  'double mass;'+\
                                  'double Pt = 0;'+\
                                  'double indx = 0;'+\
                                  'for (int i = 0; i < ntrijet; ++i) {'               +\
                                  '    if ((Pt < trijet_pt[i]) && (trijet_btag[i])) {'+\
                                  '        Pt = trijet_pt[i];'+\
                                  '        indx=i;'+\
                                  '    }'                                            +\
                                  '}'                                                +\
                                  'mass = trijet_p4[indx].M();'             +\
                                  'return mass;'
                                 )

In [1]:
mass = trijet_mass.Histo1D(('mass', 'ttbar', 25, 50, 550), 'trijet_mass')
c = TCanvas()
mass.SetFillColor(ROOT.kOrange)
mass.Draw()
c.Draw()

NameError: name 'trijet_mass' is not defined

In [72]:
import numpy as np
counts = np.array([mass.GetBinContent(i) for i in range(25)])
print(counts[1:len(counts)])
np.sum(counts[1:len(counts)])

[  82.  336.  766. 1353. 1539.  922.  626.  526.  359.  269.  212.  199.
  175.  111.   88.   94.   86.   81.   51.   49.   48.   44.   44.   28.]


8088.0

In [11]:
# rdf = jet_p4.Define('selected_
#                     ')

In [45]:
pt = trijet_pt_btag.Define('tagget_pt', 'trijet_pt[trijet_btag]').Histo1D(('pt', 'pt', 20, 25, 500),'tagget_pt')
c = TCanvas()
pt.Draw()

c.Draw()

In [48]:
for b in range(20):
    print(pt.GetBinContent(b), end=',')

7788.0,19747.0,25791.0,26238.0,22143.0,16309.0,11262.0,7612.0,5056.0,3464.0,2407.0,1738.0,1197.0,888.0,643.0,469.0,354.0,261.0,227.0,170.0,

In [27]:
import numpy as np
arr = jet_p4.AsNumpy(['jet_p4'])['jet_p4']
rvecs = np.array([arr[x].size() for x in range(len(arr))])
    

In [20]:
for p4 in arr[0]:
    print(p4.M())


24.14143943786621
14.283230781555176
17.356889724731445
8.642597198486328
8.684127807617188
6.831728458404541
5.052264213562012


In [19]:
arr = trijet_pt.AsNumpy(['trijet_pt'])['trijet_pt']
print(len(arr[0]))
for pt in arr[0]:
    print(pt)

35
121.78746121061394
137.81854090778887
161.85965155780374
194.83648634931723
166.30592139832453
78.08037539859302
129.51721293583387
89.38705758982874
109.11540288247653
88.60174087097711
79.55517550281351
74.44995597101446
120.02066419244356
126.67215448154921
115.70059860569948
83.18468474788902
45.47302589726139
59.9164610319793
44.11174586563731
55.91934300910569
109.14476161011193
74.60621770878592
66.77165382260957
23.217824641611887
90.48043864974807
167.27756857648808
127.35783847328362
155.2200043709231
110.63209582975642
150.13259558472447
98.18384114270432
86.46301429450885
114.3408648881894
86.24107857583306
57.28735107902066


In [28]:
j1, j2, j3 = arr[0][0], arr[0][1], arr[0][3]

In [30]:
print(j1.Et(), j2.Et(), j3.Et())

145.79900391360223 88.18995173825188 57.332804139070696


In [33]:
jet_pt = d.Define('selected_jets', 'jet_pt[jet_pt_mask]').AsNumpy(['selected_jets'])


In [35]:
jet_pt = jet_pt['selected_jets']

In [43]:
for pt in jet_pt[0]:
    print(pt)

144.9812469482422
87.74931335449219
80.59146118164062
57.29642868041992
45.244712829589844
27.957286834716797
26.607860565185547


In [22]:
px = 'selected_jet_px'
py = 'selected_jet_py'
pz = 'selected_jet_pz'
pt = 'selected_jet_pt'
eta = 'selected_jet_eta'
phi = 'selected_jet_phi'
mass = 'selected_jet_mass'
e = 'selected_jet_e'
jet_rdf = d.Define(px, 'jet_px[jet_pt_mask]')\
                 .Define(py, 'jet_py[jet_pt_mask]')\
                 .Define(pz, 'jet_pz[jet_pt_mask]')\
                 .Define(mass, 'jet_mass[jet_pt_mask]')\
                 .Define(pt, 'jet_pt[jet_pt_mask]')\
                 .Define(e, 'jet_e[jet_pt_mask]')\
                 .Define(eta, 'jet_eta[jet_pt_mask]')\
                 .Define(phi, 'jet_phi[jet_pt_mask]')
selected_jets = jet_rdf.AsNumpy([px, py, pz, mass, e, eta, phi, pt])

In [30]:
px = selected_jets[px]
py = selected_jets[py]
pz = selected_jets[pz]
mass = selected_jets[mass]
e = selected_jets[e]
eta = selected_jets[eta]
phi = selected_jets[phi]
pt = selected_jets[pt]

In [20]:
PxyzM4_00 = ROOT.Math.PxPyPzMVector(px[0][0], py[0][0], pz[0][0], mass[0][0])

In [31]:
LV_00 = ROOT.Math.PtEtaPhiEVector(pt[0][0], eta[0][0], phi[0][0], e[0][0])


In [32]:
print(mass[0][0], e[0][0], pt[0][0])

24.14143943786621 228.25621032714844 144.9812469482422


In [41]:
LV_00.M()

24.14140430313217

In [39]:
PxyzM4_00.Et()

145.79900391360223

In [33]:
c = TCanvas()
pt_histo = jet_rdf.Histo1D(('selected_jet_pt', 'selected jet pt', 15, 20, 200), 'selected_jet_pt')
pt_histo.Draw()
c.Draw()

In [40]:
Pt = jet_p4.Define('Pt', 'return (ROOT::VecOps::Map(jet_p4, [](ROOT::Math::PxPyPzMVector v) { return v.Pt(); }))')

In [43]:
hPt = Pt.Histo1D(('selectedjet_pt', 'selected jet pt', 15, 20, 200),'Pt')

In [44]:
c = TCanvas()
hPt.Draw()
c.Draw()